In [1]:
import numpy as np
from numpy.core.fromnumeric import argmin
import handin1 as func
import matplotlib.pyplot as plt
import matplotlib.colors as colors

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def trust_region_subproblem(gradient, hessian, lambda_0, delta, max_iter=1000):
    d = hessian.shape[0]
    lambda_j, vector = np.linalg.eig(hessian)
    lambda_1 = min(lambda_j)
    min_index = argmin(lambda_j)
    lam = lambda_0
    p = 0
    count = 0
    while lam >= -lambda_1:
        count += 1
        B_ = hessian + lam * np.identity(d)
        # try:
        if vector[:, min_index].T @ gradient == 0:
            p = hard_case(gradient, hessian, delta, lambda_j, vector, min_index)
            return p
        else:
            p = np.linalg.inv(B_) @ (-gradient)

        R = np.linalg.cholesky(B_)
        q = np.linalg.inv(R.T) @ p

        lam_new = lam + (np.linalg.norm(p) / np.linalg.norm(q)) ** 2 * (
            (np.linalg.norm(p) - delta) / delta
        )
        if lam_new == lam or (count >= max_iter and np.linalg.norm(p) <= delta):
            # return p
            break
        else:
            lam = lam_new
    return p

def hard_case(gradient, hessian, delta, lambda_j, vector, min_index):
    temp = gradient.T @ hessian @ gradient
    tau = min(1, np.linalg.norm(gradient) ** 3 / (delta * temp))
    z = vector[:, min_index]  # eigenvector of B corresponding to lamda_1
    lamda_1 = lambda_j[min_index]
    p = 0
    for i in range(np.shape(lambda_j)[0]):
        if lamda_1 != lambda_j[i]:
            p += -((vector[:, i].T @ gradient) / (lambda_j[i] - lamda_1) * vector[:, i])
    p += tau * z
    return p

def cauchy_point(gradient, hessian, delta):
    temp = gradient.T @ hessian @ gradient
    if temp <= 0:
        tau = 1
    else:
        tau = min(1, np.linalg.norm(gradient) ** 3 / (delta * temp))
    return -tau * delta / np.linalg.norm(gradient) * gradient

def trust_region(_x, fun, fun_g, fun_h, _delta, delta_max, eta=0.2, max_iter=1000):
    xs = [_x]
    deltas = [_delta]
    x = _x
    delta = _delta
    count = 0
    while count < max_iter:
        count += 1
        g = fun_g(x)
        B = fun_h(x)
        if is_pos_def(B):
            if np.linalg.norm(np.linalg.inv(B) @ g) <= delta:
                p = -np.linalg.inv(B) @ g
            else:
                p = trust_region_subproblem(g, B, 0, delta)
        else:
            p = trust_region_subproblem(g, B, -min(np.linalg.eigvals(B))+1e-5, delta)  # set the first lambda to -lambda_1

        rho = (fun(x) - fun(x + p)) / (- g.T @ p - 0.5 * p.T @ B @ p)  # should we use (4.1)? t?

        if rho < 1 / 4:
            delta = delta/4
        else:
            if rho > 3 / 4 and np.linalg.norm(p) == delta:
                delta = min(2 * delta, delta_max)

        if rho > eta:
            x = x + p
        else:
            break  # x_k = x_k+1

        xs.append(x)
        deltas.append(delta)

    return np.array(xs), fun(x), count, np.array(deltas)

def cal_rate_convergence(xs, fun):
    x_ = xs[-1]
    x1 = xs[:-1]
    x2 = xs[1:]
    res = []
    for i in range(len(x1)):
        if fun(x1[i])-fun(x_) == 0:
            continue
        res.append((fun(x2[i])-fun(x_))/(fun(x1[i])-fun(x_)))
    return np.array(res)

In [2]:
def init_x(step,start,end,dimension): #set the start&end coord, step size and dimension number pf input set
    xi=np.arange(start, end, step); x=xi #initialization of input xi for each dimension
    for i in range(dimension-1):
        x=np.vstack((np.around(x,decimals=9),np.around(xi,decimals=9))) #make x to d dimensions, from xi
    return x

def f1(x):
    alpha=1000 #set alpha
    dim=x.shape[0] #dimension number
    y=0
    for i in range(dim):
        y+=alpha**(i/(dim-1))*x[i]**2
    return y

def gradf1(x):
    alpha=1000 #set alpha
    dim=x.shape[0] #dimension number
    #dtsize=x.shape[1] #data point number
    result=np.zeros(dim)
    #print(dim,dtsize)
    for i in range(dim):
        result[i]=2*(alpha**(i/(dim-1)))*x[i]
    return result

def hessf1(x):
    alpha=1000 #set alpha
    dim=x.shape[0] #dimension number
    #dtsize=x.shape[1] #data point number
    result=np.zeros((dim,dim))
    for i in range(dim):
        result[i,i]=2*(alpha**(i/(dim-1)))
    return result

f2 = lambda x: (1-x[0])**2+100*(x[1]-x[0]**2)**2; # rosenbrok v2

def gradf2(x):
    g1 = -400*x[0]*(x[1]-x[0]**2)-2*(1-x[0])
    g2 = 200*(x[1]-x[0]**2)
    return np.array([g1, g2])

def hessf2(x):
    h11 = 2+1200*x[0]**2-400*x[1]
    h1221 = -400*x[0]
    h22 = 200
    return np.array([[h11,h1221],[h1221,h22]])

def plot2d(x0,f,f_input,fcount,f_y,name):
    x=init_x(0.01,-3.5,3.5,2)
    X1,X2 = np.meshgrid(x[0], x[1]) #generate all the data point
    dtsize=X1.shape[0] #data point number
    Y=np.zeros((dtsize,dtsize)) #initialize output results to 2D
    for i in range(dtsize):
        for j in range(dtsize):
            X=np.vstack((np.around(X1[i,j],decimals=9),np.around(X2[i,j],decimals=9))) #choose every combination of 2D inputs
            Y[i,j]=f(X) #store the results
    fx1=np.zeros(fcount+1)
    fx2=np.zeros(fcount+1)
    for i in range(fcount+1):
        if(i<=0):
            fx1[i]=x0[0]
            fx2[i]=x0[1]
        else:
            fx1[i]=f_input[i-1][0]
            fx2[i]=f_input[i-1][1]
    #plot in 2D with color
    fig, ax = plt.subplots()
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    lv=[0,1,3,10,30,100,500,2000,4000,7000,11000,14000,17000,25000]
    Cset = plt.contourf(X1, X2, Y, levels=lv,norm=colors.PowerNorm(gamma=0.25),cmap='coolwarm_r')
    #lv=[0,1,3,6,10,15,25,50,100,150,200,250,300,350,410,480,560,650,750]
    #Cset = plt.contourf(X1, X2, Y, levels=lv,norm=colors.PowerNorm(gamma=0.5),cmap='coolwarm_r')
    #Cset = plt.contourf(X1, X2, Y, levels=20,cmap='coolwarm_r')
    plt.plot(fx1, fx2,c="k")
    plt.colorbar(Cset)
    plt.title(name)
    plt.tight_layout()
    plt.savefig(name+'_2d.pdf')

#x0 = np.array([-1.8, 2.1])

#f1_input, f1_y, f1_count, f1_dels = trust_region(np.array(x0), f1, gradf1, hessf1, 0.1, 5)
#plot2d(x0,f1,f1_input,f1_count,f1_y,name="f1 start at (%.1f,%.1f)" %(x0[0],x0[1]))
#f2_input, f2_y, f2_count, f2_dels = trust_region(np.array(x0), f2, gradf2, hessf2, 0.1, 5)
#plot2d(x0,f2,f2_input,f2_count,f2_y,name="f2 start at (%.1f,%.1f) when Δ0=0.5" %(x0[0],x0[1]))
#f3_input, f3_y, f3_count, f3_dels = trust_region(np.array(x0), func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2, 0.1, 5)
#plot2d(x0,func.log_ellipsoid,f3_input,f3_count,f3_y,name="f3 start at (%.1f,%.1f)" %(x0[0],x0[1]))
#f4_input, f4_y, f4_count, f4_dels = trust_region(np.array(x0), func.f_4, func.f_4grad, func.f_4hessian, 0.1, 5)
#plot2d(x0,func.f_4,f4_input,f4_count,f4_y,name="f4 start at (%.1f,%.1f)" %(x0[0],x0[1]))
#f5_input, f5_y, f5_count, f5_dels = trust_region(np.array(x0), func.f_5, func.f_5grad, func.f_5hessian, 0.1, 5)
#plot2d(x0,func.f_5,f5_input,f5_count,f5_y,name="f5 start at (%.1f,%.1f)" %(x0[0],x0[1]))

In [3]:
'''
import handin1 as func
f3_input, f3_y, f3_count, f3_dels = trust_region(np.array(x0), func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2, 0.1, 5)
plot2d(x0,func.log_ellipsoid,f3_input,f3_count,f3_y,name="f3 start at (%.1f,%.1f)" %(x0[0],x0[1]))
'''

'\nimport handin1 as func\nf3_input, f3_y, f3_count, f3_dels = trust_region(np.array(x0), func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2, 0.1, 5)\nplot2d(x0,func.log_ellipsoid,f3_input,f3_count,f3_y,name="f3 start at (%.1f,%.1f)" %(x0[0],x0[1]))\n'

In [4]:
import math
from matplotlib.ticker import MaxNLocator

def plot_convergence(xs, fun, label="", color='b', name=""):
    # _x = xs[-1]
    # xx = cal_convergence(xs)
    xx = cal_rate_convergence(xs, fun)
    it = np.arange(xx.shape[0]-1)
    plt.plot(it, xx[:-1], c=color, label=label)
    # plt.legend()
    plt.yscale("log")
    plt.title(name)
    plt.xlabel("iteration")
    plt.ylabel("convergence rate of x")

def plot_deltas(dels, label="", name=""):
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    it = np.arange(dels.shape[0])
    #itint = range(min(it), math.ceil(max(it))+1)
    plt.plot(it, dels, label=label)
    #plt.xticks(itint)
    plt.title(name)
    plt.xlabel("Iteration")
    plt.ylabel("Δk")

def test(count,dels,func_name):
    # plot_convergence(xs, func.ellipsoid, name="convergence rate of f4")
    plot_deltas(dels, name=func_name)
    plt.savefig(func_name+".pdf")
    plt.show()
    print(count)

#f_count=np.vstack((f1_count,np.vstack((f2_count,np.vstack((f3_count,np.vstack((f4_count,f5_count))))))))
#print(f_count,"\n",f_count[:,0])
#f_dels=np.vstack((f1_dels,np.vstack((f2_dels,np.vstack((f3_dels,np.vstack((f4_dels,f5_dels))))))))

#test(f1_count, f1_dels,"f1 Δk changes")
#test(f2_count, f2_dels,"f2 Δk changes")
#test(f3_count, f3_dels,"f3 Δk changes")
#test(f4_count, f4_dels,"f4 Δk changes")
#test(f5_count, f5_dels,"f5 Δk changes")


In [11]:
import time

def opt(f,g,h):
    # time
    t=0
    # number of iteration
    i=0
    s=0
    points = 50
    for i in range(points):
        dim = 2 #point dimension number
        x = np.random.rand(dim)*5 #generate random point
        tmp1 = time.process_time()
        f_input, f_y, f_count, f_dels = trust_region(np.array(x), f, g, h, 0.1, 5)
        tmp2 = time.process_time()
        runtime=tmp2-tmp1
        t += runtime
        i += f_count
        #if np.linalg.norm(g(f_input))<=1e-6:
        #    s+=1
    print("Trust-Region &",i/points,"&",t/points)

#opt(f1, gradf1, hessf1)
opt(f2, gradf2, hessf2)
#opt(func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2)
#opt(func.f_4, func.f_4grad, func.f_4hessian)
#opt(func.f_5, func.f_5grad, func.f_5hessian)

<ipython-input-1-a1647f77ec45>:79: RuntimeWarning: invalid value encountered in double_scalars
  rho = (fun(x) - fun(x + p)) / (- g.T @ p - 0.5 * p.T @ B @ p)  # should we use (4.1)? t?
Trust-Region & 1.34 & 0.69625


In [12]:
def get_p_k(gradient, hessian, method):
    if(method == 'Newton'):
        return -np.linalg.inv(hessian) @ gradient #Newton's method
    elif(method == 'Steepest'):
        return -gradient #Steepest descent
    else:
        raise Exception('Invalid method')

def backtracking(p, x, old_x,old_alpha, old_slope, function, gradient, method): # 3.1 on page 37 and page 59
    alpha = 1
    '''
    if(method == "Steepest" and old_x is not None):
        alpha=2*(function(x)-function(old_x))/(gradient(x+0*p).T @ p)
        alpha=min(1,1.01*alpha)
        if old_slope != None:
            alpha = old_alpha * old_slope
        else:
            alpha = old_alpha
    print(alpha)
    '''
    rho = 0.8 # Good values in previous courses
    c   = 0.2 # Good values in previous courses
    while function(x + alpha * p) > function(x) + c * alpha * gradient(x).T @ p:
        alpha = rho * alpha
    return alpha

def is_converged(tolerance, x=None, oldx=None, f=None, f_grad=None, p=None, check_method='relative_x_dist'):
    if check_method == 'gradient_norm':
        return np.linalg.norm(f_grad(x)) < tolerance
    elif check_method == 'relative_x_dist':
        return oldx is not None and np.abs(f(x) - f(oldx)) < tolerance #or np.linalg.norm(x - oldx)/np.linalg.norm(oldx)
    elif check_method == 'pk':
        return np.all(np.abs(p) < tolerance)

#is_pos_def = lambda x: np.all(np.linalg.eigvals(x) > 0); #check if a matrix is positive definite or not
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def linesearch(start_point, function, gradient, hessian=None, method='Steepest', max_iter=50, tolerance=1e-03, name=""): #default configuration
    x = start_point
    old_x = None
    old_alpha = None
    old_slope = None
    old_p = None
    success = False
    idx = 0
    xs=[]; xg=[]; xf=[]; xfdivi=[]
    for idx in range(1, max_iter+1):
        hess=hessian(x)
        if(method == 'Newton'):
            if not is_pos_def(hess):
                min_diag = np.amin(hess.diagonal()) #find the minimum diagonal element of hessian(x)
                tol = 0
                if min_diag <= 0: tol = -min_diag + tolerance #change the tolerance
                d=hess.shape[0]
                hess_changed = hess + tol*np.identity(d) #adding tol times an identity matrix of size of hessian(x)
                while not is_pos_def(hess_changed):
                    tol = max(2*tol, tolerance) #decide the tol while iterating
                    hess_changed = hess + tol * np.identity(d)
            else:
                hess_changed = hess
        else:
            hess_changed = hess
        p     = get_p_k(gradient(x), hess_changed, method) # Getting direction vector
        alpha = backtracking(p, x, old_x, old_alpha, old_slope, function, gradient, method) #or use   # Getting step length
        x     = x + alpha * p # Calculating new x
        #xx = list(x)
        if(old_x is not None):
            xfdivi.append(function(x)/function(old_x))
        xf.append(function(x))
        xg.append(gradient(x))
        xs.append(x)
        if is_converged(tolerance, x, old_x, function, gradient, p, 'gradient_norm'):
            success = True
            break
        if(method == 'Steepest'): # Saving values for steepest decent method
            old_alpha = alpha
            if idx > 1:
                old_slope = (gradient(old_x).T @ old_p)/(gradient(x).T @ p)
            old_x     = x
            old_p     = p
    res = {
        'iterations': idx,
        'final_x': x,
        'success': success,
    }
    #plot_function(np.array(xfdivi), name+" "+method)
    #plot_gradient(np.array(xg), name+" "+method)
    #plot_convergence(np.array(xs), name+" "+method)
    #plot_distance(np.array(xs), name+" "+method)
    return res,xs,xf

In [19]:
def opt_line(f,g,h):
    # time
    t=0
    # number of iteration
    i=0
    points = 50
    for i in range(points):
        dim = 3 #point dimension number
        x = np.random.rand(dim)*5 #generate random point
        tmp1 = time.process_time()
        res,xs,xf = linesearch(x, f, g, h, method='Steepest', max_iter=12000, tolerance=1e-6, name="")
        tmp2 = time.process_time()
        runtime=tmp2-tmp1
        t += runtime
        i += res['iterations']
    print("Steepest &",i/points,"&",t/points)

#opt_line(f1, gradf1, hessf1)
#opt_line(f2, gradf2, hessf2)
opt_line(func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2)
#opt_line(func.f_4, func.f_4grad, func.f_4hessian)
#opt_line(func.f_5, func.f_5grad, func.f_5hessian)

Steepest & 240.98 & 22.315


In [20]:
def opt_line(f,g,h):
    # time
    t=0
    # number of iteration
    i=0
    points = 500
    for i in range(points):
        dim = 2 #point dimension number
        x = np.random.rand(dim)*5 #generate random point
        tmp1 = time.process_time()
        res,xs,xf = linesearch(x, f, g, h, method='Newton', max_iter=50, tolerance=1e-6, name="")
        tmp2 = time.process_time()
        runtime=tmp2-tmp1
        t += runtime
        i += res['iterations']
    print("Newton &",i/points,"&",t/points)

#opt_line(f1, gradf1, hessf1)
#opt_line(f2, gradf2, hessf2)
#opt_line(func.log_ellipsoid, func.log_ellipsoid_d1, func.log_ellipsoid_d2)
'''
opt_line(func.f_4, func.f_4grad, func.f_4hessian)
opt_line(func.f_5, func.f_5grad, func.f_5hessian)
'''


'\nopt_line(func.f_4, func.f_4grad, func.f_4hessian)\nopt_line(func.f_5, func.f_5grad, func.f_5hessian)\n'